In [34]:
import pandas as pd
import numpy as np
import os

In [35]:
df_ecg = pd.read_csv('data/ecg_valid.csv')
# df_ecg.columns = ['ECG']
# # get Eid from ECG file name EID_<id>.txt
# df_ecg['Eid'] = df_ecg['ECG'].apply(lambda x: int(x.split('_')[0]))
# df_ecg = df_ecg[['Eid', 'ECG']]
print(len(df_ecg))

46157


In [36]:
df_demo = pd.read_csv('/Users/natsumikyouno/UKBiobank/demographic.csv') # demographic data, age, sex, ethnicity, etc.
df_age = pd.read_csv('/Users/natsumikyouno/UKBiobank/ages.csv')
df_age.columns = ['Eid', 'age_baseline', 'age1', 'age2', 'age3'] # age at baseline, age at the first visit, age at second visit (image visit), age at the third visit
df_age = df_age[['Eid', 'age2']] # use age at the second visit (image visit) as age
df_lifestyle = pd.read_csv('/Users/natsumikyouno/UKBiobank/lifestyle/lifestyle_category.csv') # smoking, alcohol, etc.
df_physical = pd.read_csv('/Users/natsumikyouno/UKBiobank/physical_measurements.csv', usecols=['Eid', 'BMI']) # BMI, waist, hip, etc.
df_sleep = pd.read_csv('/Users/natsumikyouno/UKBiobank/lifestyle/ukb_sleep_quality_v0.csv') # sleep quality, sleep duration, etc.
df_sleep['sleep_quality_binary'] = np.where(df_sleep['sleep_quality'].isin(['Healthy']), 0, 1)
df_sleep['sleep_quality_binary'] = np.where(df_sleep['sleep_quality'].isna(), np.nan, df_sleep['sleep_quality_binary'])
df_lvf = pd.read_csv('/Users/natsumikyouno/UKBiobank/CMR/lv_functions_preprocessed.csv') # LV functions, LVEF, LVEDV, LVESV, etc.
df_cvd_events = pd.read_csv('data/CVD_events.csv') # CVD events, time to event
# biomarkers, SBP, DBP, CRP, HbA1c, LDL, HDL, Triglycerides, Glucose, Cholesterol
df_biomarkers = pd.read_csv('/Users/natsumikyouno/UKBiobank/biomarkers.csv', usecols=['Eid', 'SBP', 'DBP', 'CRP', 'HbA1c', 'Glucose', 'LDLc', 'HDLc', 'TG', 'CHOL'])
df_med = pd.read_csv('/Users/natsumikyouno/UKBiobank/ukb_medication_processed.csv', usecols=['Eid', 'medication0', 'medication1', 'medication2', 'medication3'])

In [37]:
df = pd.merge(df_ecg, df_demo, on='Eid', how='left')
df = pd.merge(df, df_age, on='Eid', how='left')
df = pd.merge(df, df_physical, on='Eid', how='left')
df = pd.merge(df, df_sleep, on='Eid', how='left')
df = pd.merge(df, df_biomarkers, on='Eid', how='left')
df = pd.merge(df, df_lvf, on='Eid', how='left')
df = pd.merge(df, df_lifestyle, on='Eid', how='left')
df = pd.merge(df, df_med, on='Eid', how='left')
df = pd.merge(df, df_cvd_events, on='Eid', how='left')

In [38]:
df['Age_Baseline'] = df['Age']
df['Age'] = df['age2'] # use age at the second visit (image visit) as age
df['TyG'] = np.log(df['Glucose'] * 88.57 * df['TG'] * 18 / 2)

In [39]:
# count missing values
for col in df.columns:
    n_miss = df[col].isna().sum()
    pct_miss = n_miss / len(df) * 100
    print(f'{col}: {df[col].isna().sum()} ({pct_miss:.2f}%)')

Eid: 0 (0.00%)
ECG: 0 (0.00%)
Sex: 0 (0.00%)
Age: 0 (0.00%)
Ethnicity_code: 15 (0.03%)
Ethnicity: 0 (0.00%)
TDI: 45 (0.10%)
IMD: 1104 (2.39%)
Income: 4626 (10.02%)
Income_v1: 38896 (84.27%)
Income_v2: 4727 (10.24%)
Income_v3: 42037 (91.07%)
Education: 0 (0.00%)
Education_v1: 0 (0.00%)
Education_v2: 0 (0.00%)
Education_v3: 0 (0.00%)
age2: 0 (0.00%)
BMI: 2 (0.00%)
sleep_duration: 113 (0.24%)
chronotype: 5466 (11.84%)
insomnia: 34 (0.07%)
snoring: 2588 (5.61%)
dozing: 88 (0.19%)
sleep_time: 0 (0.00%)
sleep_duration_score: 113 (0.24%)
chronotype_score: 5466 (11.84%)
insomnia_score: 34 (0.07%)
snoring_score: 2588 (5.61%)
dozing_score: 88 (0.19%)
sleep_quality_score: 7758 (16.81%)
sleep_quality: 7758 (16.81%)
sleep_quality_binary: 7758 (16.81%)
SBP: 413 (0.89%)
DBP: 413 (0.89%)
CHOL: 3059 (6.63%)
CRP: 3154 (6.83%)
Glucose: 6855 (14.85%)
HbA1c: 3391 (7.35%)
HDLc: 6811 (14.76%)
LDLc: 3154 (6.83%)
TG: 3094 (6.70%)
LVEF: 11720 (25.39%)
LVEDV: 11720 (25.39%)
LVESV: 11720 (25.39%)
LVSV: 11720 (25.

In [40]:
df.to_csv('data/ukb_ecg_data.csv', index=False)

In [41]:
# remove subjects with cvd events
df_no_cvd_before = df[~((df['last_event'] == 1) & (df['last_time'] > -10))] # subjects without CVD event in last 10 years
print(df_no_cvd_before.shape[0])
df_no_cvd_before.to_csv('data/ukb_ecg_data_no_cvd_before.csv', index=False)

43100


In [42]:
df_no_mediation = df_no_cvd_before[df_no_cvd_before['medication2'] == 0] # remove subjects with diabetes medication
print(df_no_mediation.shape[0])
df_no_mediation.to_csv('data/ukb_ecg_data_selected.csv', index=False)

34884


In [43]:
df_nona = df[~(df['sleep_quality_binary'].isna())]
print(df_nona['sleep_quality_binary'].value_counts())

sleep_quality_binary
1.0    22821
0.0    15578
Name: count, dtype: int64


In [44]:
df_nona['cvd'].value_counts()

cvd
atrial_fibrillation          804
chronic_artery_disease       522
angina                       306
stroke                       182
heart_failure                106
peripheral_artery_disease     97
Name: count, dtype: int64

In [45]:
df_nona['event'].value_counts()

event
0    36382
1     2017
Name: count, dtype: int64

In [46]:
df_no_mediation['event'].value_counts()

event
0    33598
1     1286
Name: count, dtype: int64

In [47]:
df_no_mediation['cvd'].value_counts()

cvd
atrial_fibrillation          523
chronic_artery_disease       274
angina                       186
stroke                       160
peripheral_artery_disease     77
heart_failure                 66
Name: count, dtype: int64

event
0    34176
1     1676
Name: count, dtype: int64